# 03 Model vs Human Gaze Comparison
 
Load model-predicted saliency metrics and human-percentile scores, compute mean±SEM,
and display a summary table.

In [17]:
import pickle
import math
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# Choose dataset: 'memento' or 'videomem'
dataset = 'memento'

# Paths for raw metric scores and human-percentile scores
dirs = {
    'memento': dict(
        all_scores='./pickles/model_human_metrics/FINAL_memento_metrics_gaussianBeforeRescale.pkl',
        auc_percentiles='./pickles/model_human_metrics/FINAL_memento_percentiles_all.pkl'
    ),
    'videomem': dict(
        all_scores='./pickles/model_human_metrics/FINAL_videomem_metrics_gaussianBeforeRescale.pkl',
        auc_percentiles='./pickles/model_human_metrics/FINAL_videomem_percentiles_all.pkl'
    )
}
paths = dirs[dataset]

In [19]:
# Load dictionaries
with open(paths['all_scores'], 'rb') as f:
    all_dict = pickle.load(f)
with open(paths['auc_percentiles'], 'rb') as f:
    auc_pct = pickle.load(f)

In [20]:
# Wrap human percentiles under 'AUC_P'
auc_dict = {'AUC_P': auc_pct}

### Compute mean and SEM

In [21]:
def mean_sem(values):
    """Return (mean, sem) for list or dict values."""
    if isinstance(values, dict):
        vals = [v for v in values.values() if not pd.isna(v)]
    else:
        vals = [v for v in values if not pd.isna(v)]
    m = sum(vals) / len(vals)
    std = math.sqrt(sum((x-m)**2 for x in vals) / len(vals))
    return m, std / math.sqrt(len(vals))

In [22]:
metrics = ['AUC_Judd', 'AUC_P', 'NSS', 'CC', 'KLD']
results = []
for met in metrics:
    if met == 'AUC_P':
        data = auc_dict.get(met, {})
    else:
        data = all_dict.get(met, {})
    if data:
        m, se = mean_sem(data)
        results.append({'Metric': met, 'M-H': f"{m:.2f} ± {se:.2f}"})

### Build DataFrame

In [23]:
df_summary = pd.DataFrame(results).set_index('Metric')
df_summary.index = ['AUC-J ↑', 'AUC-P ↑', 'NSS ↑', 'CC ↑', 'KLD ↓']

In [24]:
plt.figure(figsize=(6,2))
sns.set_style('whitegrid')
print(f"Dataset: {dataset.title()}")
display(df_summary)

Dataset: Videomem


,M-H
AUC-J ↑,0.89 ± 0.01
AUC-P ↑,88.88 ± 1.28
NSS ↑,2.00 ± 0.07
CC ↑,0.27 ± 0.01
KLD ↓,2.65 ± 0.02


<Figure size 600x200 with 0 Axes>